In [9]:
import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

boardsize = (9,6)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
imgObjects = {}


In [10]:

images = glob.glob('resizedChessboardOk/*[0-9].jpg')
i = 0
for fname in images:
    i = i + 1
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, boardsize,None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)

#         cv2.imwrite(f'resizedChessboardOk/{i}.jpg', img)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, boardsize, corners,ret)
        cv2.imwrite(fname.replace('.jpg', '_cor.jpg'), img)
        
        imgObjects[fname] = img
#         cv2.imshow(f'img-{fname}',img)
#         cv2.waitKey(1000)
    
        
    else:
        print(f'cannot find corners for {fname}')

# cv2.destroyAllWindows()

In [11]:
for fname, img in imgObjects.items():
    cv2.imshow('img', img)
    cv2.waitKey(5000)
    cv2.destroyAllWindows()

In [12]:
# Calibrate camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [13]:
print(ret)

0.1849233650491676


In [14]:
print(mtx)

[[972.95923425   0.         608.78583476]
 [  0.         972.29283463 455.24308471]
 [  0.           0.           1.        ]]


In [15]:
# reprojection error
tot_error = 0.
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error
#     print(f'error for {i}: {error}')

print("total error: ", tot_error/len(objpoints))

total error:  0.023852566485823283


In [16]:
np.save('mtx.npy', mtx)
np.save('dist.npy', dist)